In [8]:
import tkinter as tk
from tkinter import ttk
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import numpy as np


def draw_grid(rows, cols, mode, ax):
    """
    Рисует прямоугольник, разбитый на подпрямоугольники с раскрашиванием и нумерацией.

    :param rows: Количество делений по вертикали.
    :param cols: Количество делений по горизонтали.
    :param mode: Режим раскраски: 'spiral' или 'snake'.
    :param ax: Matplotlib Axes для рисования.
    """
    # Размеры общего прямоугольника
    width, height = cols, rows

    # Создаем палитру цветов ("радуга")
    colors = plt.cm.rainbow(np.linspace(0, 1, rows * cols))

    # Функция для получения индексов по спирали
    def spiral_indices(rows, cols):
        spiral = []
        left, right, top, bottom = 0, cols - 1, 0, rows - 1
        while left <= right and top <= bottom:
            # Верхняя строка
            for col in range(left, right + 1):
                spiral.append((top, col))
            top += 1

            # Правая колонка
            for row in range(top, bottom + 1):
                spiral.append((row, right))
            right -= 1
            # Нижняя строка
            if top <= bottom:
                for col in range(right, left - 1, -1):
                    spiral.append((bottom, col))
                bottom -= 1

            # Левая колонка
            if left <= right:
                for row in range(bottom, top - 1, -1):
                    spiral.append((row, left))
                left += 1

        return spiral

    # Функция для получения индексов "змейкой"
    def snake_indices(rows, cols):
        snake = []
        for row in range(rows):
            if row % 2 == 0:
                # Слева направо
                for col in range(cols):
                    snake.append((row, col))
            else:
                # Справа налево
                for col in range(cols - 1, -1, -1):
                    snake.append((row, col))
        return snake

    # Получаем индексы в зависимости от режима
    if mode == 'spiral':
        indices = spiral_indices(rows, cols)
    elif mode == 'snake':
        indices = snake_indices(rows, cols)
    else:
        raise ValueError("Unsupported mode. Choose 'spiral' or 'snake'.")

    # Очищаем axes перед рисованием
    ax.clear()
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_aspect('equal')
    ax.axis('off')

    # Рисуем прямоугольники и добавляем номера
    color_idx = 0
    for i, (row, col) in enumerate(indices):
        x, y = col, rows - row - 1  # Преобразуем индексы в координаты
        rect = plt.Rectangle((x, y), 1, 1, color=colors[color_idx])
        ax.add_patch(rect)
        
        # Добавляем номер внутри квадрата
        ax.text(x + 0.5, y + 0.5, str(i + 1), ha='center', va='center', fontsize=8, color='black')

        color_idx += 1


def update_grid():
    """Обновляет график."""
    try:
        rows = int(rows_entry.get())
        cols = int(cols_entry.get())
        mode = mode_combobox.get()
        draw_grid(rows, cols, mode, ax)
        canvas.draw()
    except ValueError:
        print("Please enter valid numbers for rows and columns.")

# Настройка Tkinter окна
window = tk.Tk()
window.title("Grid Drawer")

# Фрейм для ввода параметров
input_frame = ttk.Frame(window)
input_frame.pack(pady=10)

# Метки и поля ввода для строк
ttk.Label(input_frame, text="Rows:").grid(row=0, column=0, padx=5)
rows_entry = ttk.Entry(input_frame)
rows_entry.grid(row=0, column=1, padx=5)

# Метки и поля ввода для колонок
ttk.Label(input_frame, text="Cols:").grid(row=1, column=0, padx=5)
cols_entry = ttk.Entry(input_frame)
cols_entry.grid(row=1, column=1, padx=5)


# Метка и выпадающий список для режима
ttk.Label(input_frame, text="Mode:").grid(row=2, column=0, padx=5)
mode_combobox = ttk.Combobox(input_frame, values=['spiral', 'snake'])
mode_combobox.grid(row=2, column=1, padx=5)
mode_combobox.set('spiral')

# Кнопка для обновления
update_button = ttk.Button(input_frame, text="Update", command=update_grid)
update_button.grid(row=3, column=0, columnspan=2, pady=10)

# Создание фигуры matplotlib и добавление её в Tkinter
fig = Figure(figsize=(6, 4), dpi=100)
ax = fig.add_subplot(111)
canvas = FigureCanvasTkAgg(fig, master=window)
canvas_widget = canvas.get_tk_widget()
canvas_widget.pack()


# Запуск Tkinter главного цикла
window.mainloop()